In [27]:
!python --version

Python 3.10.11


In [28]:
import torch
import numpy as np
import pandas as pd
import sklearn as learn
import matplotlib as plt
import pydantic
import time


In [29]:
print(np.__version__)
print(torch.__version__)
print(pd.__version__)
print(learn.__version__)
print(plt.__version__)
print(pydantic.__version__)
print('hello')

1.24.3
2.0.1
2.0.3
1.3.0
3.7.2
2.1.1
hello


In [30]:
torch.cuda.get_arch_list()

['sm_37',
 'sm_50',
 'sm_60',
 'sm_61',
 'sm_70',
 'sm_75',
 'sm_80',
 'sm_86',
 'compute_37']

In [31]:
for i in range(torch.cuda.device_count()):
   print(torch.cuda.get_device_properties(i).name)


NVIDIA GeForce GTX 960M


In [32]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [33]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [34]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [35]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [36]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [37]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [38]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [39]:
epochs = 10
start = time.time()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
stop = time.time()
print(stop - start)

Epoch 1
-------------------------------
loss: 2.311163  [   64/60000]
loss: 2.294985  [ 6464/60000]
loss: 2.278280  [12864/60000]
loss: 2.274232  [19264/60000]
loss: 2.250019  [25664/60000]
loss: 2.236108  [32064/60000]
loss: 2.233609  [38464/60000]
loss: 2.205967  [44864/60000]
loss: 2.203595  [51264/60000]
loss: 2.178077  [57664/60000]
Test Error: 
 Accuracy: 47.9%, Avg loss: 2.170032 

Epoch 2
-------------------------------
loss: 2.180527  [   64/60000]
loss: 2.170811  [ 6464/60000]
loss: 2.116231  [12864/60000]
loss: 2.131195  [19264/60000]
loss: 2.075464  [25664/60000]
loss: 2.031966  [32064/60000]
loss: 2.048534  [38464/60000]
loss: 1.979299  [44864/60000]
loss: 1.976770  [51264/60000]
loss: 1.910845  [57664/60000]
Test Error: 
 Accuracy: 58.2%, Avg loss: 1.910121 

Epoch 3
-------------------------------
loss: 1.942414  [   64/60000]
loss: 1.917210  [ 6464/60000]
loss: 1.796360  [12864/60000]
loss: 1.832455  [19264/60000]
loss: 1.719087  [25664/60000]
loss: 1.677275  [32064/600